In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/

Mounted at /content/drive
/content/drive/MyDrive


In [ ]:
%pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.2.34 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 30.2/78.2 GB disk)


In [ ]:
import torch
import torch.nn as nn
from torch import nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import models, transforms
import torchvision.transforms as transforms
from transformers import BertTokenizer, BertModel
from ultralytics import YOLO
from collections import defaultdict
from PIL import Image
from tqdm import tqdm
import json
import csv
import os

In [ ]:
# Ensure CUDA (GPU support) is available if possible, else use CPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')

Using device: cuda


In [ ]:
# Load the trained YOLOv8 model
yolo_model = YOLO('/content/drive/MyDrive/00_PFE/Object_Detection/Training_Results/Yolov8-V6/Results/runs/train/experiment/weights/best.pt').to(device)

In [ ]:
# Define the label mapping
label_mapping = [
    "flooded", "non flooded", "flooded,non flooded", "Yes", "No",
    "0", "1", "2", "3", "4", "5", "6", "7", "8", "9",
    "10", "11", "12", "13", "14", "15", "16", "17", "18", "19",
    "20", "21", "22", "23", "24", "25", "26", "27", "28", "29",
    "30", "31", "32", "33", "34", "35", "36", "37", "38", "39",
    "40", "41", "42", "43", "44", "45", "46", "47", "48", "49", "50"
]

In [ ]:
# Function to extract features from YOLOv8
def extract_yolo_features(image_path, model, device):
    results = model(image_path)

    # Initialize lists to store extracted features
    boxes_list = []
    conf_list = []
    cls_list = []

    for result in results:
        if result.boxes is not None:
            boxes = result.boxes.xyxy.to(device)  # Bounding box coordinates
            confs = result.boxes.conf.to(device)  # Confidence scores
            classes = result.boxes.cls.to(device)  # Class values
            boxes_list.append(boxes)
            conf_list.append(confs)
            cls_list.append(classes)

    # Combine features into a single tensor
    if boxes_list:
        features = torch.cat([torch.cat(boxes_list), torch.cat(conf_list).unsqueeze(1), torch.cat(cls_list).unsqueeze(1)], dim=1)
    else:
        features = torch.empty((0, 6), device=device)

    return features

In [ ]:
# VQADataset class
class VQADataset(Dataset):
    def __init__(self, annotations_file, img_dir, tokenizer, transform=None):
        with open(annotations_file, 'r') as f:
            self.annotations = json.load(f)
        self.img_dir = img_dir
        self.transform = transform
        self.tokenizer = tokenizer
        self.img_to_annotations = self._group_by_image()

    def _group_by_image(self):
        img_to_annotations = defaultdict(list)
        for idx, annotation in self.annotations.items():
            img_to_annotations[annotation['Image_ID']].append(annotation)
        return img_to_annotations

    def __len__(self):
        return len(self.img_to_annotations)

    def __getitem__(self, idx):
        image_id = list(self.img_to_annotations.keys())[idx]
        annotations = self.img_to_annotations[image_id]
        img_path = os.path.join(self.img_dir, image_id)
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        questions = []
        answers = []
        for annotation in annotations:
            inputs = self.tokenizer.encode_plus(
                annotation['Question'],
                add_special_tokens=True,
                return_tensors='pt',
                padding='max_length',
                truncation=True,
                max_length=64
            )
            question = inputs['input_ids'].squeeze(0).to(device)
            attention_mask = inputs['attention_mask'].squeeze(0).to(device)
            answer_text = str(annotation['Ground_Truth'])
            answer_idx = label_mapping.index(answer_text)
            questions.append((question, attention_mask))
            answers.append(torch.tensor(answer_idx, device=device))
        return {
            'image_path': img_path,
            'questions': questions,
            'attention_masks': [am for _, am in questions],
            'answers': torch.stack(answers)
        }

In [ ]:
def custom_collate_fn(batch):
    batch_image_paths = [item['image_path'] for item in batch]
    batch_questions = [q for item in batch for q, _ in item['questions']]
    batch_attention_masks = [am for item in batch for _, am in item['questions']]
    batch_answers = torch.cat([item['answers'] for item in batch])
    num_questions_per_image = [len(item['questions']) for item in batch]
    return {
        'image_paths': batch_image_paths,
        'questions': batch_questions,
        'attention_masks': batch_attention_masks,
        'answers': batch_answers,
        'num_questions_per_image': num_questions_per_image
    }

In [ ]:
class VQAModel(nn.Module):
    def __init__(self, bert_model, yolo_input_dim, hidden_dim, vocab_size):
        super(VQAModel, self).__init__()
        self.bert_model = bert_model
        self.fc_yolo = nn.Linear(yolo_input_dim, hidden_dim)
        self.lstm = nn.LSTM(input_size=hidden_dim + 768, hidden_size=hidden_dim, num_layers=2, batch_first=True)
        self.fc_out = nn.Linear(hidden_dim, vocab_size)
        self.hidden_dim = hidden_dim

    def forward(self, image_features, questions, attention_masks, num_questions_per_image):
        image_features = [self.fc_yolo(image_feature) for image_feature in image_features]
        image_features = torch.stack(image_features)

        text_features = [self.bert_model(question.unsqueeze(0).to(image_features.device), attention_mask=attention_mask.unsqueeze(0).to(image_features.device)).pooler_output for question, attention_mask in zip(questions, attention_masks)]
        text_features = torch.cat(text_features, dim=0)

        expanded_image_features = []
        for image_feature, num_questions in zip(image_features, num_questions_per_image):
            expanded_image_features.append(image_feature.repeat(num_questions, 1))

        expanded_image_features = torch.cat(expanded_image_features, dim=0)

        if expanded_image_features.size(0) != text_features.size(0):
            raise RuntimeError(f"Mismatch in feature sizes: {expanded_image_features.size(0)} vs {text_features.size(0)}")

        combined_features = torch.cat((expanded_image_features, text_features), dim=1).unsqueeze(1)

        h0 = torch.zeros(2, combined_features.size(0), self.hidden_dim).to(combined_features.device)
        c0 = torch.zeros(2, combined_features.size(0), self.hidden_dim).to(combined_features.device)

        outputs, _ = self.lstm(combined_features, (h0, c0))
        outputs = outputs.squeeze(1)
        logits = self.fc_out(outputs)

        return logits

In [ ]:
# Initialize tokenizer, BERT model, and VQA model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased').to(device)
num_classes = len(label_mapping)
hidden_dim = 256
vqa_model = VQAModel(bert_model=bert_model, yolo_input_dim=6, hidden_dim=hidden_dim, vocab_size=num_classes).to(device)

In [ ]:
# Define optimizer and loss function
optimizer = torch.optim.Adam(vqa_model.parameters(), lr=0.0001)
criterion = torch.nn.CrossEntropyLoss()

In [ ]:
# Initialize dataset and dataloader
annotations_file = '/content/drive/MyDrive/00_PFE/DataSet/Visual_Question_Answering /FloodNet Challenge @ EARTHVISION 2021 - Track 2/Questions/Training Question.json'
img_dir = '/content/drive/MyDrive/00_PFE/DataSet/Visual_Question_Answering /FloodNet Challenge @ EARTHVISION 2021 - Track 2/Images/Train_Image'
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])
dataset = VQADataset(annotations_file, img_dir, tokenizer, transform)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True, collate_fn=custom_collate_fn)

In [ ]:
# Setup logging
def setup_logging():
    with open('log.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Epoch", "Average Loss", "Average Accuracy"])

def log_epoch(epoch, avg_loss, avg_accuracy):
    with open('log.csv', 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([epoch, f"{avg_loss:.4f}", f"{avg_accuracy * 100:.2f}%"])

In [ ]:
# Function to compute accuracy
def compute_accuracy(predictions, labels):
    _, predicted = torch.max(predictions, 1)
    correct = (predicted == labels).float().sum()
    return correct / labels.size(0)

In [ ]:
# Training loop
num_epochs = 8
setup_logging()

for epoch in range(num_epochs):
    total_loss = 0
    total_accuracy = 0
    total_batches = 0
    vqa_model.train()
    with tqdm(dataloader, desc=f"Epoch {epoch + 1}") as pbar:
        for batch in pbar:
            image_paths = batch['image_paths']
            questions = batch['questions']
            attention_masks = batch['attention_masks']
            answers = batch['answers']
            num_questions_per_image = batch['num_questions_per_image']

            # Extract features using YOLOv8
            image_features_list = []
            for image_path in image_paths:
                features = extract_yolo_features(image_path, yolo_model, device)
                if features.size(0) == 0:
                    features = torch.zeros(1, 6, device=device)  # Handling no detections case
                image_features_list.append(features.mean(dim=0))
            image_features = torch.stack(image_features_list)

            optimizer.zero_grad()
            outputs = vqa_model(image_features, questions, attention_masks, num_questions_per_image)
            loss = criterion(outputs, answers)
            if torch.isnan(loss):
                print(f"Encountered NaN loss, skipping this batch")
                continue
            loss.backward()
            torch.nn.utils.clip_grad_norm_(vqa_model.parameters(), max_norm=2.0)  # Gradient clipping
            optimizer.step()

            # Calculate loss and accuracy for the batch
            batch_loss = loss.item()
            batch_accuracy = compute_accuracy(outputs, answers).item()

            # Aggregate information for overall epoch statistics
            total_loss += batch_loss
            total_accuracy += batch_accuracy
            total_batches += 1

            # Update the progress bar with the current loss and accuracy
            pbar.set_postfix(Loss=batch_loss, Accuracy=f"{batch_accuracy:.4f}")

    # Display average loss and accuracy for the epoch
    avg_loss = total_loss / total_batches
    avg_accuracy = total_accuracy / total_batches
    print(f"Epoch {epoch + 1} - Average Loss: {avg_loss:.4f}, Average Accuracy: {avg_accuracy:.4f}")

    # Log epoch data to CSV
    log_epoch(epoch + 1, avg_loss, avg_accuracy)

    # Saving the model state dictionary
    torch.save(vqa_model.state_dict(), f"/content/drive/MyDrive/00_PFE/VQA/Code-V2/VQAModel_Epoch_{epoch+1}.pth")

print("Training complete!")

Epoch 1:   0%|          | 0/23 [00:00<?, ?it/s]


image 1/1 /content/drive/MyDrive/00_PFE/DataSet/Visual_Question_Answering /FloodNet Challenge @ EARTHVISION 2021 - Track 2/Images/Train_Image/7263.JPG: 448x640 13 Building-floodeds, 1 Road-flooded, 7.5ms
Speed: 2.8ms preprocess, 7.5ms inference, 1.5ms postprocess per image at shape (1, 3, 448, 640)

image 1/1 /content/drive/MyDrive/00_PFE/DataSet/Visual_Question_Answering /FloodNet Challenge @ EARTHVISION 2021 - Track 2/Images/Train_Image/7977.JPG: 480x640 (no detections), 7.5ms
Speed: 2.9ms preprocess, 7.5ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /content/drive/MyDrive/00_PFE/DataSet/Visual_Question_Answering /FloodNet Challenge @ EARTHVISION 2021 - Track 2/Images/Train_Image/6734.JPG: 480x640 3 Building-non-floodeds, 8.5ms
Speed: 3.0ms preprocess, 8.5ms inference, 1.7ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /content/drive/MyDrive/00_PFE/DataSet/Visual_Question_Answering /FloodNet Challenge @ EARTHVISION 2021 - Track 2/Images/

Epoch 1:   4%|▍         | 1/23 [01:47<39:15, 107.06s/it, Accuracy=0.1421, Loss=3.99]


image 1/1 /content/drive/MyDrive/00_PFE/DataSet/Visual_Question_Answering /FloodNet Challenge @ EARTHVISION 2021 - Track 2/Images/Train_Image/6657.JPG: 480x640 6 Building-non-floodeds, 2 Road-non-floodeds, 7.4ms
Speed: 3.1ms preprocess, 7.4ms inference, 1.7ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /content/drive/MyDrive/00_PFE/DataSet/Visual_Question_Answering /FloodNet Challenge @ EARTHVISION 2021 - Track 2/Images/Train_Image/7164.JPG: 480x640 (no detections), 6.9ms
Speed: 3.0ms preprocess, 6.9ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /content/drive/MyDrive/00_PFE/DataSet/Visual_Question_Answering /FloodNet Challenge @ EARTHVISION 2021 - Track 2/Images/Train_Image/8431.JPG: 480x640 (no detections), 6.8ms
Speed: 3.0ms preprocess, 6.8ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /content/drive/MyDrive/00_PFE/DataSet/Visual_Question_Answering /FloodNet Challenge @ EARTHVISION 2021 - Track 2/Images/

Epoch 1:   9%|▊         | 2/23 [03:34<37:33, 107.33s/it, Accuracy=0.1895, Loss=3.97]

...


image 1/1 /content/drive/MyDrive/00_PFE/DataSet/Visual_Question_Answering /FloodNet Challenge @ EARTHVISION 2021 - Track 2/Images/Train_Image/7939.JPG: 480x640 (no detections), 7.4ms
Speed: 3.1ms preprocess, 7.4ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /content/drive/MyDrive/00_PFE/DataSet/Visual_Question_Answering /FloodNet Challenge @ EARTHVISION 2021 - Track 2/Images/Train_Image/8465.JPG: 480x640 2 Road-non-floodeds, 7.6ms
Speed: 3.4ms preprocess, 7.6ms inference, 2.4ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /content/drive/MyDrive/00_PFE/DataSet/Visual_Question_Answering /FloodNet Challenge @ EARTHVISION 2021 - Track 2/Images/Train_Image/8958.JPG: 480x640 1 Road-non-flooded, 7.2ms
Speed: 3.2ms preprocess, 7.2ms inference, 1.8ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /content/drive/MyDrive/00_PFE/DataSet/Visual_Question_Answering /FloodNet Challenge @ EARTHVISION 2021 - Track 2/Images/Train_Image/6708.JPG: 

Epoch 8:  96%|█████████▌| 22/23 [14:25<00:39, 39.08s/it, Accuracy=0.4824, Loss=1.88]


image 1/1 /content/drive/MyDrive/00_PFE/DataSet/Visual_Question_Answering /FloodNet Challenge @ EARTHVISION 2021 - Track 2/Images/Train_Image/8085.JPG: 480x640 1 Road-non-flooded, 6.9ms
Speed: 2.9ms preprocess, 6.9ms inference, 1.8ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /content/drive/MyDrive/00_PFE/DataSet/Visual_Question_Answering /FloodNet Challenge @ EARTHVISION 2021 - Track 2/Images/Train_Image/10175.JPG: 480x640 2 Road-non-floodeds, 6.7ms
Speed: 2.9ms preprocess, 6.7ms inference, 1.6ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /content/drive/MyDrive/00_PFE/DataSet/Visual_Question_Answering /FloodNet Challenge @ EARTHVISION 2021 - Track 2/Images/Train_Image/8143.JPG: 480x640 1 Road-non-flooded, 6.7ms
Speed: 3.0ms preprocess, 6.7ms inference, 1.9ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /content/drive/MyDrive/00_PFE/DataSet/Visual_Question_Answering /FloodNet Challenge @ EARTHVISION 2021 - Track 2/Images/Train_Image/9063.J

Epoch 8: 100%|██████████| 23/23 [14:50<00:00, 38.71s/it, Accuracy=0.4672, Loss=1.97]


Epoch 8 - Average Loss: 2.0761, Average Accuracy: 0.4346
Training complete!


In [ ]:
model_path = '/content/drive/MyDrive/00_PFE/VQA/Code-V2/VQAModel_Epoch_8.pth'
if os.path.exists(model_path):
    vqa_model.load_state_dict(torch.load(model_path))
    print("Loaded the trained model.")

Loaded the trained model.


In [ ]:
vqa_model.eval()

VQAModel(
  (bert_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [ ]:
# Testing the model
vqa_model.eval()
test_annotations_file = '/content/drive/MyDrive/00_PFE/DataSet/Visual_Question_Answering /FloodNet Challenge @ EARTHVISION 2021 - Track 2/Questions/Test Question.json'
test_img_dir = '/content/drive/MyDrive/00_PFE/DataSet/Visual_Question_Answering /FloodNet Challenge @ EARTHVISION 2021 - Track 2/Images/Test_Image'
test_dataset = VQADataset(test_annotations_file, test_img_dir, tokenizer, transform)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False, collate_fn=custom_collate_fn)

correct_predictions = 0
total_predictions = 0

with torch.no_grad():
    for batch in tqdm(test_dataloader, desc="Testing"):
        image_paths = batch['image_paths']
        questions = batch['questions']
        attention_masks = batch['attention_masks']
        answers = batch['answers']
        num_questions_per_image = batch['num_questions_per_image']

        # Extract features using YOLOv8
        image_features_list = []
        for image_path in image_paths:
            features = extract_yolo_features(image_path, yolo_model, device)
            if features.size(0) == 0:
                features = torch.zeros(1, 6, device=device)  # Handling no detections case
            image_features_list.append(features.mean(dim=0))
        image_features = torch.stack(image_features_list)

        outputs = vqa_model(image_features, questions, attention_masks, num_questions_per_image)
        _, predicted = torch.max(outputs, 1)
        correct_predictions += (predicted == answers).sum().item()
        total_predictions += answers.size(0)

accuracy = correct_predictions / total_predictions
print(f"Test Accuracy: {accuracy * 100:.2f}%")

In [ ]:
# Inference function
def infer(image_path, question):
    # Load and preprocess the image
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(device)

    # Extract features using YOLOv8
    with torch.no_grad():
        features = extract_yolo_features(image_path, yolo_model, device).mean(dim=0).unsqueeze(0)

    # Preprocess the question
    inputs = tokenizer.encode_plus(
        question,
        add_special_tokens=True,
        return_tensors='pt',
        padding='max_length',
        truncation=True,
        max_length=64
    )
    question_input = inputs['input_ids'].squeeze(0).to(device)
    attention_mask = inputs['attention_mask'].squeeze(0).to(device)

    # Pass through the model
    with torch.no_grad():
        num_questions_per_image = [1]
        outputs = vqa_model(features, [question_input], [attention_mask], num_questions_per_image)
        _, predicted = torch.max(outputs, 1)

    # Map the prediction to the label
    answer = label_mapping[predicted.item()]
    return answer

In [ ]:
# Example usage
image_path = '/content/drive/MyDrive/00_PFE/VQA/Image_Predction_VQA/5.jpg'  # Update with your image path
question = "Is the entire road non flooded?"

answer = infer(image_path, question)
print(f"Question: {question}")
print(f"Answer: {answer}")


image 1/1 /content/drive/MyDrive/00_PFE/VQA/Image_Predction_VQA/5.jpg: 448x640 15 Building-floodeds, 2 Road-floodeds, 174.3ms
Speed: 3.9ms preprocess, 174.3ms inference, 5.0ms postprocess per image at shape (1, 3, 448, 640)
Question: Is the entire road non flooded?
Answer: non flooded
